In [1]:
!pip install -e ..

Obtaining file:///C:/Users/hunkb/Downloads/quantum-chemistry-workshop-bell-state
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for quantum_chemistry (pyproject.toml): started
  Building editable for quantum_chemistry (pyproject.toml): finished with status 'done'
  Created wheel for quantum_chemistry: filename=quantum_chemistry-0.0.1-0.editable-py3-none-any.whl size=3709 sha256=a5833edcad521ce22be614765648cc369f7a9b1d210ac3042bc7f7bbe3fdf3ee
  Stored in directory: C:\Users\hunkb\AppData\Local\Temp\pip-ephem-wheel-cache-bkqz6q

In [2]:
import numpy as np
from IPython.display import display, Markdown # For italicized print statements
%load_ext autoreload
%autoreload 0

Notebook by **Maxime Dion** <maxime.dion@usherbrooke.ca><br>
For the QSciTech-QuantumBC virtual workshop on gate-based quantum computing

# Important

When you modify and save a `*.py` file you need to re-import it so that your modifications can be taken into account when you re-execute a call. By adding the magic command `%autoreload` at the beginning of a cell, you make sure that the modifications you did to the `*.py` files are taken into account when you re-run a cell and that you can see the effect.

If you encounter weird behavior, restart the kernel and try again.

# Tutorial 1 (Mapping)

In this tutorial you will complete the implementation of the classes `PauliString` and `Operator` which will be useful to build a qubit representation of the Hamiltonian of a small molecule. You will then implement the Jordan Wigner mapping.

By completing all sections of this notebook you will be able to :
- Create `PauliString` instances.
- Multiply `PauliString` together.
- Translate a `PauliString` into a unitary matrix of size `(2**n)**2`.
- Create `Operator` instances.
- Multiply and add `Operator` together.
- Combine repeated `PauliString`s in `Operator`.
- Translate a `Operator` into a matrix of size `(2**n)**2`.
- Use the Jordan-Wigner mapping to translate a Fermionic Hamiltonian into a Qubit Hamiltonian as an instance of an `Operator`.

The solution we suggest here is NOT mandatory. If you find ways to make it better and more efficient, go on and impress us! 

**Remark on qiskit**

We use `qiskit` in this workshop. The tools you'll be building here are already available in `qiskit`. For instance, the `PauliString` and `Operator` classes here have similar purpuse to `Pauli` and `SparsePauliOp` in `qiskit`. Nevertheless, we strongly encourage you to complete the current implementation because we think it as a valuable learning experience.

At the end of the workshop, we encourage you to use `qiksit` to complete the same project. It should give you the same results, but now you'll understand what is going on under the hood.

# PauliString

The `PauliString` class is partially implemented in the file `Pauli.py`.

In [3]:
from quantum_chemistry.pauli import PauliString

## Creation

This object's attributes are 2 arrays of booleans `z_bits` and `x_bits`. You can easily create an instance and print the result. The `__str__` method is already implemented so you can use `print()` on any instance of `PauliString`. The boolean arrays in input are in the `0123` order and the label string is in the reversed `q3q2q1q0` order. Here we initialize the Pauli string `YXZI`.

In [4]:
def show_pauli_mapping(z_bits, x_bits):
    """Display the relationship between z_bits, x_bits, and Pauli operators."""
    print("\nPauli Operator Mapping Table:")
    print("-" * 50)
    print(f"{'Qubit':<8} {'z_bit':<8} {'x_bit':<8} {'Operator':<10} {'Meaning'}")
    print("-" * 50)
    
    # Process each qubit position to determine corresponding Pauli operator
    for i in range(len(z_bits)):
        z, x = z_bits[i], x_bits[i]
        if z == 0 and x == 0:
            op = 'I'
            meaning = 'Identity'
        elif z == 1 and x == 0:
            op = 'Z'
            meaning = 'Z rotation'
        elif z == 0 and x == 1:
            op = 'X'
            meaning = 'X rotation'
        elif z == 1 and x == 1:
            op = 'Y'
            meaning = 'Y rotation (Z and X)'
        
        print(f"{i:<8} {int(z):<8} {int(x):<8} {op:<10} {meaning}")
    
    print("-" * 50)
    
    # Construct the Pauli string representation
    ops = []
    for i in range(len(z_bits)):
        z, x = z_bits[i], x_bits[i]
        if z == 0 and x == 0: ops.append('I')
        elif z == 1 and x == 0: ops.append('Z')
        elif z == 0 and x == 1: ops.append('X')
        elif z == 1 and x == 1: ops.append('Y')
    
    # Create both internal and display order strings
    pauli_str = ''.join(ops)
    reversed_str = ''.join(reversed(ops))
    
    # Print results and explanation
    print(f"\nPauli string (internal order q0,q1,q2,q3): {pauli_str}")
    display(Markdown("*NOTE: The displayed string is reversed compared to the bit arrays!*"))
    
# Use case example:
z_bits = np.array([0, 1, 0, 1], dtype=bool)
x_bits = np.array([0, 0, 1, 1], dtype=bool)
show_pauli_mapping(z_bits, x_bits)
pauli_string = PauliString(z_bits, x_bits)
print('Pauli string: ', pauli_string)


Pauli Operator Mapping Table:
--------------------------------------------------
Qubit    z_bit    x_bit    Operator   Meaning
--------------------------------------------------
0        0        0        I          Identity
1        1        0        Z          Z rotation
2        0        1        X          X rotation
3        1        1        Y          Y rotation (Z and X)
--------------------------------------------------

Pauli string (internal order q0,q1,q2,q3): IZXY


*NOTE: The displayed string is reversed compared to the bit arrays!*

Pauli string:  YXZI


### Creation exercice

Create the `ZZXY` Pauli string. Remember that the arrays are in the 0...n order, but the string representation in the reverse order. The `print` should return `ZZXY`.

In [5]:
z_bits = np.array([1, 0, 1, 1], dtype=bool)
x_bits = np.array([1, 1, 0, 0], dtype=bool)
show_pauli_mapping(z_bits, x_bits)
print('Pauli string: ', PauliString(z_bits, x_bits))


Pauli Operator Mapping Table:
--------------------------------------------------
Qubit    z_bit    x_bit    Operator   Meaning
--------------------------------------------------
0        1        1        Y          Y rotation (Z and X)
1        0        1        X          X rotation
2        1        0        Z          Z rotation
3        1        0        Z          Z rotation
--------------------------------------------------

Pauli string (internal order q0,q1,q2,q3): YXZZ


*NOTE: The displayed string is reversed compared to the bit arrays!*

Pauli string:  ZZXY


### Creation from string

To create `PauliString` using alternative input data, you can implement `@classmethod`. 

Such a method called `from_str` is partially implemented in the `PauliString` class. Complete this implementation so that you can use it to create `PauliString` from a string such as `YXZI`.

In [6]:
%autoreload
pauli_string = PauliString.from_str('YXZI')
print('Pauli string: ', pauli_string)
display(Markdown("*NOTE: This allows us to generate Pauli strings from ascii strings and not binary strings!*"))


Pauli string:  YXZI


*NOTE: This allows us to generate Pauli strings from ascii strings and not binary strings!*

### Useful methods

In order to compare `PauliString`s together it's convenient to represent it as `zx_bits` which is an array twice as long that combines `z_bits` and `x_bits`. Implement the `to_zx_bits()` method. Why not do the `to_xz_bits()` while your at it!

You should get :

<code>
[False  True False  True False False  True  True]<br>[False False  True  True False  True False  True]
</code>

In [7]:
%autoreload
pauli_string = PauliString.from_str('YXZI')
print('Pauli string: ', pauli_string)
zx_bits = pauli_string.to_zx_bits()
xz_bits = pauli_string.to_xz_bits()
print('ZX Bits: ', zx_bits)
print('XZ Bits: ', xz_bits)

Pauli string:  YXZI
ZX Bits:  [False  True False  True False False  True  True]
XZ Bits:  [False False  True  True False  True False  True]


It's also useful to know where are the $\hat{I}$ in a `PauliString`. Implement the method that does this. You should get 

<code>[ True False False False]</code>

In [8]:
%autoreload
pauli_string = PauliString.from_str('YXZI')
ids = pauli_string.ids() # Checks for identity operators in the string
print(ids)
display(Markdown("*NOTE: The string representation is reversed, so for IZXY the boolean array holds true!*"))

[ True False False False]


*NOTE: The string representation is reversed, so for IZXY the boolean array holds true!*

## Multiplication with another PauliString

Multiplying `PauliString`s is essential to be able to translate Fermionic Hamiltonians into a qubit Hamiltonian. 

Before you implement the method that will allow you to do this, you should experiment a bit with how boolean arrays behave. Take a look at methods like `np.dot()`, `np.logical_and()`, `np.logical_or()` and `np.logical_xor()`. In particular, notice that the addition `+` on booleans is not a (mod 2) addition (it's a `logical_or`) and the `np.sum()` method on a boolean array counts the number of 1 and returns an `int`.

In [9]:
# Initialize boolean arrays for demonstration
bits_1 = np.array([0,1,0,1], dtype = bool)
bits_2 = np.array([0,1,1,1], dtype = bool) # NOT ocupation numbers, just bit mathematics!

# Show raw arrays
print("Original boolean arrays:")
print(f"bits_1: {bits_1}")
print(f"bits_2: {bits_2}")

# Demonstrate operations
print("\nBoolean operations:")
print(f"Simple addition (+): {bits_1 + bits_2}")
print(f"Count of True values (np.sum) in bits_1: {np.sum(bits_1)}")
print(f"Logical AND: {np.logical_and(bits_1, bits_2)}")
print(f"Logical OR:  {np.logical_or(bits_1, bits_2)}")
print(f"Logical XOR: {np.logical_xor(bits_1, bits_2)}  <- This is mod 2 addition!")
print(f"Dot product: {np.dot(bits_1, bits_2)}")

display(Markdown("""
*IMPORTANT: When working with boolean arrays:*
* Regular addition (`+`) performs logical OR, not modular addition
* For mod 2 addition (needed for Pauli operations), use `np.logical_xor()`
* `np.sum()` counts the number of True values, returning an integer
"""))

Original boolean arrays:
bits_1: [False  True False  True]
bits_2: [False  True  True  True]

Boolean operations:
Simple addition (+): [False  True  True  True]
Count of True values (np.sum) in bits_1: 2
Logical AND: [False  True False  True]
Logical OR:  [False  True  True  True]
Logical XOR: [False False  True False]  <- This is mod 2 addition!
Dot product: True



*IMPORTANT: When working with boolean arrays:*
* Regular addition (`+`) performs logical OR, not modular addition
* For mod 2 addition (needed for Pauli operations), use `np.logical_xor()`
* `np.sum()` counts the number of True values, returning an integer


With these considerations, implement the `mul_pauli_string()` method in order to replicate the product

\begin{align}
\hat{I}\hat{Y}\hat{Z}\hat{Z} \times \hat{I}\hat{I}\hat{X}\hat{Z}  = i \hat{I}\hat{Y}\hat{Y}\hat{I}.
\end{align}

The product return a `PauliString` and a phase (`complex`). The method `__mul__` is already implemeted to call `mul_pauli_string()` so you can use `*` to do the product.

In [10]:
%autoreload
pauli_string_1 = PauliString.from_str('IYZZ')
pauli_string_2 = PauliString.from_str('IIXZ')
new_pauli_string, phase = pauli_string_1 * pauli_string_2
print('Resulting Pauli string: ', new_pauli_string)
print('Resulting phase: ', phase)
display(Markdown("*NOTE: The multiplication of Pauli strings is not commutative!*"))

Resulting Pauli string:  IYYI
Resulting phase:  1j


*NOTE: The multiplication of Pauli strings is not commutative!*

Check your solution on many pairs of Pauli strings such as

\begin{align}
\hat{Z}\hat{Z}\hat{Z}\hat{Z} \times \hat{X}\hat{X}\hat{X}\hat{I}  = -i \hat{Y}\hat{Y}\hat{Y}\hat{Z}.
\end{align}

In [11]:
%autoreload
pauli_string_1 = PauliString.from_str('ZZZZ')
pauli_string_2 = PauliString.from_str('XXXI')
new_pauli_string, phase = pauli_string_1 * pauli_string_2
print('Resulting Pauli string: ', new_pauli_string)
print('Resulting phase: ', phase)
display(Markdown("*NOTE: The multiplication of Pauli strings is not commutative!*"))

Resulting Pauli string:  YYYZ
Resulting phase:  (-0-1j)


*NOTE: The multiplication of Pauli strings is not commutative!*

## Matrix representation
The matrix reprensetation of `PauliString` will only be used to compute the exact solution of the Hamiltonian. It will not be used for quantum computing, but it's a nice way to validate your results.

Any `PauliString` can be converted into a matrix. This is useful to find the exact solution for small systems. To combine the space of two qubits we use the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) ($\otimes$). For example, the `ZX` Pauli string can be represented as the following matrix

\begin{align}
    \hat{Z}_1\hat{X}_0 = \hat{Z}_1\otimes\hat{X}_0 &= \begin{pmatrix} 1 \times \hat{X}_0 & 0 \\ 0 & -1 \times \hat{X}_0 \end{pmatrix} \\
    &= \begin{pmatrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & -1 \\ 0 & 0 & -1 & 0 \end{pmatrix}
\end{align}

which is expressed in the basis

\begin{align}
    |00\rangle, |01\rangle, |10\rangle, |11\rangle.
\end{align} 

Indeed we verify that

\begin{align}
    \hat{Z}_1\hat{X}_0 |10\rangle &= - |11\rangle \\
    \begin{pmatrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & -1 \\ 0 & 0 & -1 & 0 \end{pmatrix}\begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \end{pmatrix} &= -\begin{pmatrix} 0 \\ 0 \\ 0 \\ 1 \end{pmatrix}
\end{align}

The `np.kron()` method is a straight foward way to acheive this. The previous 2 `PauliString`s can be turned into a $4\times 4$ matrix like this.

In [12]:
z_matrix = np.array([[1, 0], [0, -1]], dtype = int)
x_matrix = np.array([[0, 1], [1, 0]], dtype = int)
zx_matrix = np.kron(z_matrix, x_matrix)

def print_matrix(matrix):
    """Display a matrix in a more readable format"""
    # Set numpy print options
    with np.printoptions(precision=2, suppress=True, linewidth=100):
        # Print each row on its own line for clarity
        for row in matrix:
            print("[", end=" ")
            for val in row:
                if abs(val.imag) < 1e-10:  # Just show real part if imaginary is ~0
                    print(f"{val.real:5.2f}", end=" ")
                else:
                    print(f"{val:5.2f}", end=" ")
            print("]")

# Example usage:
print("Z matrix:")
print_matrix(z_matrix)
print("\nX matrix:")
print_matrix(x_matrix)
print("\nZX matrix (Kronecker product):")
print_matrix(zx_matrix)

Z matrix:
[  1.00  0.00 ]
[  0.00 -1.00 ]

X matrix:
[  0.00  1.00 ]
[  1.00  0.00 ]

ZX matrix (Kronecker product):
[  0.00  1.00  0.00  0.00 ]
[  1.00  0.00  0.00  0.00 ]
[  0.00  0.00  0.00 -1.00 ]
[  0.00  0.00 -1.00  0.00 ]


Implement the `to_matrix()` method for any Pauli string and try it to find the matrix form of `ZX`.

In [13]:
%autoreload
pauli_string = PauliString.from_str('ZX')
print(pauli_string)
matrix = pauli_string.to_matrix()
print_matrix(matrix)

ZX
[  0.00  0.00  1.00  0.00 ]
[  0.00 -0.00  0.00 -1.00 ]
[  1.00  0.00  0.00  0.00 ]
[  0.00 -1.00  0.00 -0.00 ]


# Operator
The `Operator` class is partially implemented in the file `Pauli.py`.

In [14]:
from quantum_chemistry.pauli import Operator

## Creation

To build a `Operator` you only need to provide an `numpy.array` of coefficients (`complex`) and a `numpy.array` of `PauliString`. If they are not arrays, they will be converted. Here again the `__str__()` method is already implemented.

In [15]:
# Create coefficients and Pauli strings to form an Operator
coefs = np.array([0.5, 0.5], dtype=complex)
pauli_string_1 = PauliString.from_str('IIXZ')
pauli_string_2 = PauliString.from_str('IYZZ')
pauli_strings = np.array([pauli_string_1, pauli_string_2], dtype=PauliString)

# Show the coefficients
print(f"\nCoefficients array (shape {coefs.shape}):")
for i, coef in enumerate(coefs):
    print(f"  coef[{i}] = {coef}")

# Show the Pauli strings
print("\nPauli strings:")
for i, ps in enumerate(pauli_strings):
    print(f"  pauli_string[{i}] = {ps}")
    # For each Pauli string, show its internal representation
    print(f"    z_bits: {ps.z_bits}")
    print(f"    x_bits: {ps.x_bits}")

# Create and display the operator
operator = Operator(coefs, pauli_strings)
print("\nResulting quantum operator (linear combination):")
print(f"  {operator}")

# Show mathematical form
display(Markdown(r"*Mathematical representation:* $\hat{O} = \sum_i c_i \hat{P}_i$"))
display(Markdown(r"*In this case:* $\hat{O} = 0.5 \cdot \hat{I}\hat{I}\hat{X}\hat{Z} + 0.5 \cdot \hat{I}\hat{Y}\hat{Z}\hat{Z}$"))


Coefficients array (shape (2,)):
  coef[0] = (0.5+0j)
  coef[1] = (0.5+0j)

Pauli strings:
  pauli_string[0] = IIXZ
    z_bits: [ True False False False]
    x_bits: [False  True False False]
  pauli_string[1] = IYZZ
    z_bits: [ True  True  True False]
    x_bits: [False False  True False]

Resulting quantum operator (linear combination):
  (0.50+0.00j)*IIXZ + (0.50+0.00j)*IYZZ


*Mathematical representation:* $\hat{O} = \sum_i c_i \hat{P}_i$

*In this case:* $\hat{O} = 0.5 \cdot \hat{I}\hat{I}\hat{X}\hat{Z} + 0.5 \cdot \hat{I}\hat{Y}\hat{Z}\hat{Z}$

### Multiplication of a PauliString by a coefficient

Multiplying a `PauliString` by a number is a useful way to create a `Operator` with only 1 `PauliString`. Implement the method `mul_coef` in the `PauliString` class so that you can easily create a `Operator`.

In [16]:
%autoreload
operator_pauli = 1 * PauliString.from_str('IIXZ')
print(operator_pauli)

(1.00)*IIXZ


### Adding of Operators
The sum of two `Operator`s is just the union of these two ensembles. Implement `add_operator()` and test your solution here. The `__add__()` method is already implemented to call `add_operator()` so you can use the `+` operator.

In [17]:
%autoreload
operator = 0.5*pauli_string_1 + 0.5*pauli_string_2
print(operator)

(0.50)*IIXZ + (0.50)*IYZZ


### Product of Operators

The product of two `Operator`s can be computed using the distributive property of any sum. While you implement `mul_operator()` make sure you take into account the phase coming from the product of two `PauliString`s. You can test your code on the following cell.

In [18]:
%autoreload
operator_1 = 1*PauliString.from_str('IIXZ')
operator_2 = 1*PauliString.from_str('IYZZ')
new_operator = operator_1 * operator_2
print(new_operator)

(0.00-1.00j)*IYYI


You should get:

<code>
(0.00-1.00j)*IYYI
</code>

With addition and multiplication, `Operator`s are much more convenient to work with than `PauliString`s because they carry the possible phase from the product.

## Accessing subset of an Operator
A `__getitem__()` method is already implemented to access subset of the `Operator`. You can use indices and slices, like a `list` or an `np.array`.

In [19]:
# Create an operator with four Pauli strings
operator = 1*PauliString.from_str('IIIZ') + 1*PauliString.from_str('IIZI') + 1*PauliString.from_str('IZII') + 1*PauliString.from_str('ZIII')
print(f"\nComplete operator: {operator}")

# Demonstrate indexing and slicing
print("\nAccessing individual terms:")
print(f"operator[0] = {operator[0]}  # First term")
print(f"operator[1:3] = {operator[1:3]}  # Slice from index 1 to 2")
print(f"operator[-1] = {operator[-1]}  # Last term")

# Show mathematical form
display(Markdown(r"""
*Mathematical representation:*
* Full operator: $\hat{O} = \hat{Z}_0\hat{I}_1\hat{I}_2\hat{I}_3 + \hat{I}_0\hat{Z}_1\hat{I}_2\hat{I}_3 + \hat{I}_0\hat{I}_1\hat{Z}_2\hat{I}_3 + \hat{I}_0\hat{I}_1\hat{I}_2\hat{Z}_3$
* First term: $\hat{O}[0] = \hat{Z}_0\hat{I}_1\hat{I}_2\hat{I}_3$
* Slice: $\hat{O}[1:3] = \hat{I}_0\hat{Z}_1\hat{I}_2\hat{I}_3 + \hat{I}_0\hat{I}_1\hat{Z}_2\hat{I}_3$
* Last term: $\hat{O}[-1] = \hat{I}_0\hat{I}_1\hat{I}_2\hat{Z}_3$
"""))


Complete operator: (1.00)*IIIZ + (1.00)*IIZI + (1.00)*IZII + (1.00)*ZIII

Accessing individual terms:
operator[0] = (1.00)*IIIZ  # First term
operator[1:3] = (1.00)*IIZI + (1.00)*IZII  # Slice from index 1 to 2
operator[-1] = (1.00)*ZIII  # Last term



*Mathematical representation:*
* Full operator: $\hat{O} = \hat{Z}_0\hat{I}_1\hat{I}_2\hat{I}_3 + \hat{I}_0\hat{Z}_1\hat{I}_2\hat{I}_3 + \hat{I}_0\hat{I}_1\hat{Z}_2\hat{I}_3 + \hat{I}_0\hat{I}_1\hat{I}_2\hat{Z}_3$
* First term: $\hat{O}[0] = \hat{Z}_0\hat{I}_1\hat{I}_2\hat{I}_3$
* Slice: $\hat{O}[1:3] = \hat{I}_0\hat{Z}_1\hat{I}_2\hat{I}_3 + \hat{I}_0\hat{I}_1\hat{Z}_2\hat{I}_3$
* Last term: $\hat{O}[-1] = \hat{I}_0\hat{I}_1\hat{I}_2\hat{Z}_3$


## Combinaison and threshold
When a `PauliString` is present many times in a `Operator`, it is convenient to be able to remove extra occurences by combining the respective coefficients. Let's take the example from the presentation.

In [20]:
operator_1 = 1*PauliString.from_str('IIIZ') - 0.5*PauliString.from_str('IIZZ') 
operator_2 = 1*PauliString.from_str('ZZZI') + 0.5*PauliString.from_str('ZZII') 
operator_3 = operator_1 * operator_2
print(operator_3)

(1.00)*ZZZZ + (0.50)*ZZIZ + (-0.50)*ZZIZ + (-0.25)*ZZZZ


We see that `ZZZZ` occurs 2 times and `ZZIZ` occurs 2 times as well.

Implement the `combine()` method to reduce the `Operator` to 2 `PauliString`s. There are many ways to do that. Suggestion, convert with `to_zx_bits()` and use the `np.unique()` method. DO NOT remove `PauliString`s with `0` coef yet.

In [21]:
%autoreload
operator_combined = operator_3.combine()
print(operator_combined)

(0.00+0.00j)*ZZIZ + (0.75+0.00j)*ZZZZ

You should get:

<code>
(0.75+0.00j)*ZZZZ + (0.00+0.00j)*ZZIZ
</code>

Implement the `apply_threshold()` method to get rid of any Pauli string with a coefficient smaller than the `threshold`.

In [22]:
%autoreload
operator = operator_combined.apply_threshold() # Threshold of 1e-10
print(operator)

(0.75+0.00j)*ZZZZ


You should get:

<code>
(0.75+0.00j)*ZZZZ
</code>

## Matrix representation

The `Operator` can also be represented as a matrix. This matrix is just the linear combinaison of the matrices representing each Pauli string. Implement the `to_matrix()` method.

In [23]:
%autoreload
small_operator = 1*PauliString.from_str('ZZ') + 2*PauliString.from_str('XX')
matrix = small_operator.to_matrix()
print_matrix(matrix)

[  1.00  0.00  0.00  2.00 ]
[  0.00 -1.00  2.00  0.00 ]
[  0.00  2.00 -1.00  0.00 ]
[  2.00  0.00  0.00  1.00 ]


You should get :

<code>
[[ 1.+0.j  0.+0.j  0.+0.j  2.+0.j]<br> [ 0.+0.j -1.+0.j  2.+0.j  0.+0.j]<br> [ 0.+0.j  2.+0.j -1.+0.j  0.+0.j]<br> [ 2.+0.j  0.+0.j  0.+0.j  1.+0.j]] 
</code>

# Molecular Hamiltonian

Before constructing the qubit representation of the molecular Hamiltonian we need the Hamiltonian in its fermionic representation. Therefore, we first need to get the integral tensors. We will use the pre-computed integrals from the `h2_data` directory.


### Loading the integrals from files

The integrals are precomputed for a given set of interatomic distances. To load the data for one distance use the following code. The first ouput is the distance. The next two are the 2d and 4d tensors for the one body and two body integrals. The last is the nuclear repulsion energy which will contribute to the total energy.

In [24]:
from quantum_chemistry.molecule.h2_molecule import load_h2_spin_orbital_integral

distance, one_body, two_body, nuc_eneg = load_h2_spin_orbital_integral("../h2_data","h2_mo_integrals_d_0750.npz")

Loading h2_mo_integrals_d_0750.npz


You can also load them all at one. It will return you a list of distances and a list of tuples structured such this way `(one_body, two_body, nuc_eneg)`. 

In [25]:
from quantum_chemistry.molecule.h2_molecule import load_h2_spin_orbital_integrals

distances, molecule_datas = load_h2_spin_orbital_integrals("../h2_data")

Loading h2_mo_integrals_d_0300.npz
Loading h2_mo_integrals_d_0350.npz
Loading h2_mo_integrals_d_0400.npz
Loading h2_mo_integrals_d_0450.npz
Loading h2_mo_integrals_d_0500.npz
Loading h2_mo_integrals_d_0550.npz
Loading h2_mo_integrals_d_0600.npz
Loading h2_mo_integrals_d_0650.npz
Loading h2_mo_integrals_d_0700.npz
Loading h2_mo_integrals_d_0750.npz
Loading h2_mo_integrals_d_0800.npz
Loading h2_mo_integrals_d_0850.npz
Loading h2_mo_integrals_d_0900.npz
Loading h2_mo_integrals_d_0950.npz
Loading h2_mo_integrals_d_1000.npz
Loading h2_mo_integrals_d_1050.npz
Loading h2_mo_integrals_d_1100.npz
Loading h2_mo_integrals_d_1150.npz
Loading h2_mo_integrals_d_1200.npz
Loading h2_mo_integrals_d_1250.npz
Loading h2_mo_integrals_d_1300.npz
Loading h2_mo_integrals_d_1350.npz
Loading h2_mo_integrals_d_1400.npz
Loading h2_mo_integrals_d_1450.npz
Loading h2_mo_integrals_d_1500.npz
Loading h2_mo_integrals_d_1550.npz
Loading h2_mo_integrals_d_1600.npz
Loading h2_mo_integrals_d_1650.npz
Loading h2_mo_integr

# Mapping
You are now in good position to implement your first mapping. The functions to perform the Jordan-Wigner Mapping are partially implemented in the file `Mapping.py`.

## Jordan-Wigner creation/annihilation operators

The first task of the mapping is to translate creation and annihilation fermionic operators into `Operator`. You now need to implement the `creation_annihilation_operators_with_jordan_wigner()` function. It should return 2 lists of `Operator`s, one `list` for creation operators and one `list` for annihilation operators. You can make use of the addition and multipliation method you implemented earlier.

Refer to the presentation for the general structure of the Jordan-Wigner mapping. Make sure your method works for different numbers of qubits.

In [26]:
%autoreload
from quantum_chemistry.mapping import creation_annihilation_operators_with_jordan_wigner

creation_operators, annihilation_operators = creation_annihilation_operators_with_jordan_wigner(4)
print(len(creation_operators), 'creation operators')
print('Creation operators')
for ap in creation_operators:
    print(ap)
print()
print(len(annihilation_operators), 'annihilation operators')
print('Annihilation operators')
for am in annihilation_operators:
    print(am)

from IPython.display import display, Markdown
display(Markdown(r"""
*Mathematical form of Jordan-Wigner mapping:*

For creation operators:
$$a_p^\dagger = \frac{1}{2}(X_p - iY_p) \prod_{j=0}^{p-1}Z_j$$

For annihilation operators:
$$a_p = \frac{1}{2}(X_p + iY_p) \prod_{j=0}^{p-1}Z_j$$

*Explicit expressions for each operator:*

$$a_0^\dagger = \frac{1}{2}(X_0 - iY_0)$$
$$a_1^\dagger = \frac{1}{2}(X_1 - iY_1)Z_0$$
$$a_2^\dagger = \frac{1}{2}(X_2 - iY_2)Z_0Z_1$$
$$a_3^\dagger = \frac{1}{2}(X_3 - iY_3)Z_0Z_1Z_2$$
"""))

4 creation operators
Creation operators
(0.50+0.00j)*IIIX + (-0.00-0.50j)*IIIY
(0.50+0.00j)*IIXZ + (-0.00-0.50j)*IIYZ
(0.50+0.00j)*IXZZ + (-0.00-0.50j)*IYZZ
(0.50+0.00j)*XZZZ + (-0.00-0.50j)*YZZZ

4 annihilation operators
Annihilation operators
(0.50+0.00j)*IIIX + (0.00+0.50j)*IIIY
(0.50+0.00j)*IIXZ + (0.00+0.50j)*IIYZ
(0.50+0.00j)*IXZZ + (0.00+0.50j)*IYZZ
(0.50+0.00j)*XZZZ + (0.00+0.50j)*YZZZ



*Mathematical form of Jordan-Wigner mapping:*

For creation operators:
$$a_p^\dagger = \frac{1}{2}(X_p - iY_p) \prod_{j=0}^{p-1}Z_j$$

For annihilation operators:
$$a_p = \frac{1}{2}(X_p + iY_p) \prod_{j=0}^{p-1}Z_j$$

*Explicit expressions for each operator:*
$$a_0^\dagger = \frac{1}{2}(X_0 - iY_0)$$
$$a_1^\dagger = \frac{1}{2}(X_1 - iY_1)Z_0$$
$$a_2^\dagger = \frac{1}{2}(X_2 - iY_2)Z_0Z_1$$
$$a_3^\dagger = \frac{1}{2}(X_3 - iY_3)Z_0Z_1Z_2$$


For the creation operators you should get.

<code>
4 creation operators<br>
Creation operators<br>
(0.50+0.00j)*IIIX + (-0.00-0.50j)*IIIY<br>
(0.50+0.00j)*IIXZ + (-0.00-0.50j)*IIYZ<br>
(0.50+0.00j)*IXZZ + (-0.00-0.50j)*IYZZ<br>
(0.50+0.00j)*XZZZ + (-0.00-0.50j)*YZZZ<br>
</code>

For the annihilation just reverse the sign of the imaginary part.

## Building the Qubit Hamiltonian

The construct the qubit Hamiltonian, you only need to use the mapped creation and annihilation operators and compose them following the Hamiltonien formulation.

### One body term

Let's start with the one_body part. The one body Hamiltonian is of the form 

\begin{align*}
    \hat{H}_1 = \sum_{i,j} h_{ij} \hat{a}_i^\dagger \hat{a}_j
\end{align*} 

You should now be able to implement the `build_one_body_qubit_hamiltonian` function in the `mapping.py` file. 

Each fermionic operator is made of 2 Pauli strings. So each term $i,j$ creates 4 Pauli strings. There is 16 terms in $h_{ij}$ so there is 64 Pauli strings (if we do not consider combinaison and applying threshold yet). Your implementation should now return a `Operator` of length 64.

In [36]:
%autoreload
from quantum_chemistry.mapping import build_one_body_qubit_hamiltonian, creation_annihilation_operators_with_jordan_wigner
from quantum_chemistry.molecule.h2_molecule import load_h2_spin_orbital_integral

# Load molecular data for H2 at a bond distance of 0.75 angstroms
distance, one_body, two_body, nuc_eneg = load_h2_spin_orbital_integral("../h2_data","h2_mo_integrals_d_0750.npz")
print(f"Bond distance: {distance} Å")
print(f"Nuclear repulsion energy: {nuc_eneg} hartree")
print(f"One-body tensor shape: {one_body.shape}")

# Print a sample of the one-body integral values
print("\nSample of one-body integral values (h_ij):")
for i in range(min(4, one_body.shape[0])):
    for j in range(min(4, one_body.shape[1])):
        if abs(one_body[i, j]) > 1e-10:
            print(f"  h_{i},{j} = {one_body[i, j].real:.6f}")

# Create the creation and annihilation operators using Jordan-Wigner mapping
creation_operators, annihilation_operators = creation_annihilation_operators_with_jordan_wigner(4)
print(f"\nCreated {len(creation_operators)} creation operators and {len(annihilation_operators)} annihilation operators")
print("First creation operator:")
print(f"  {creation_operators[0]}")
print("First annihilation operator:")
print(f"  {annihilation_operators[0]}")

# Build the one-body qubit Hamiltonian
print("\nBuilding one-body qubit Hamiltonian...")
one_body_operator = build_one_body_qubit_hamiltonian(one_body, creation_operators, annihilation_operators)
print(f"Raw one-body operator has {len(one_body_operator.coefs)} Pauli terms")

# Display mathematical representations of the equations
from IPython.display import display, Markdown
display(Markdown(r"""
*The one-body Hamiltonian terms are expressed mathematically as:*

$$\hat{H}_1 = \sum_{i,j} h_{ij} \hat{a}_i^\dagger \hat{a}_j$$

Where:
- $h_{ij}$ are the one-body integrals (shown in the sample above)
- $\hat{a}_i^\dagger$ is the creation operator for orbital $i$
- $\hat{a}_j$ is the annihilation operator for orbital $j$

*The Jordan-Wigner mapping transforms these fermionic operators into Pauli strings:*

$$\hat{a}_p^\dagger = \frac{1}{2}(\hat{X}_p - i\hat{Y}_p) \prod_{j=0}^{p-1}\hat{Z}_j$$

$$\hat{a}_p = \frac{1}{2}(\hat{X}_p + i\hat{Y}_p) \prod_{j=0}^{p-1}\hat{Z}_j$$
"""))

Loading h2_mo_integrals_d_0750.npz
Bond distance: 0.75 Å
Nuclear repulsion energy: 0.70556961456 hartree
One-body tensor shape: (4, 4)

Sample of one-body integral values (h_ij):
  h_0,0 = -1.247285
  h_1,1 = -0.481273
  h_2,2 = -1.247285
  h_3,3 = -0.481273

Created 4 creation operators and 4 annihilation operators
First creation operator:
  (0.50+0.00j)*IIIX + (-0.00-0.50j)*IIIY
First annihilation operator:
  (0.50+0.00j)*IIIX + (0.00+0.50j)*IIIY

Building one-body qubit Hamiltonian...
Raw one-body operator has 16 Pauli terms



*The one-body Hamiltonian terms are expressed mathematically as:*

$$\hat{H}_1 = \sum_{i,j} h_{ij} \hat{a}_i^\dagger \hat{a}_j$$

Where:
- $h_{ij}$ are the one-body integrals (shown in the sample above)
- $\hat{a}_i^\dagger$ is the creation operator for orbital $i$
- $\hat{a}_j$ is the annihilation operator for orbital $j$

*The Jordan-Wigner mapping transforms these fermionic operators into Pauli strings:*

$$\hat{a}_p^\dagger = \frac{1}{2}(\hat{X}_p - i\hat{Y}_p) \prod_{j=0}^{p-1}\hat{Z}_j$$

$$\hat{a}_p = \frac{1}{2}(\hat{X}_p + i\hat{Y}_p) \prod_{j=0}^{p-1}\hat{Z}_j$$


In [39]:
# Display the mathematical explanation of the one-body operator processing
display(Markdown(r"""
### Processing the One-body Operator

We apply several operations to simplify the raw one-body operator:

1. **apply_threshold()** - Removes terms with near-zero coefficients
2. **combine()** - Merges duplicate Pauli strings by adding their coefficients
3. **apply_threshold()** - Removes any new near-zero terms created during combination
4. **sort()** - Orders terms by coefficient magnitude for readability

This process transforms our raw operator with many terms into a simplified expression with only the significant terms.
"""))

# Apply the processing pipeline to the one-body operator
one_body_operator = one_body_operator.apply_threshold().combine().apply_threshold().sort()
display(Markdown(r"""
### Simplifying the One-Body Operator

The raw Hamiltonian contains many terms with near-zero coefficients and duplicate Pauli strings.
We apply several operations to simplify it:

1. **apply_threshold()** - Removes terms with coefficients below 1e-10
2. **combine()** - Consolidates identical Pauli strings by adding their coefficients
3. **apply_threshold()** - Removes any new terms with negligible coefficients
4. **sort()** - Orders terms by their coefficients' absolute values.

The resulting operator is a more compact representation of the form:

$$\hat{H}_1 = \sum_j c_j \hat{P}_j$$

Where:
- $c_j$ is the real coefficient for each term
- $\hat{P}_j$ is a Pauli string (product of I, X, Y, Z operators)

This more efficient representation helps us implement the Hamiltonian on quantum hardware with fewer gates.
"""))
print(one_body_operator)


### Processing the One-body Operator

We apply several operations to simplify the raw one-body operator:

1. **apply_threshold()** - Removes terms with near-zero coefficients
2. **combine()** - Merges duplicate Pauli strings by adding their coefficients
3. **apply_threshold()** - Removes any new near-zero terms created during combination
4. **sort()** - Orders terms by coefficient magnitude for readability

This process transforms our raw operator with many terms into a simplified expression with only the significant terms.



### Processing the One-body Operator

We apply several operations to simplify the raw one-body operator:

1. **apply_threshold()** - Removes terms with near-zero coefficients
2. **combine()** - Merges duplicate Pauli strings by adding their coefficients
3. **apply_threshold()** - Removes any new near-zero terms created during combination
4. **sort()** - Orders terms by coefficient magnitude for readability

This process transforms our raw operator with many terms into a simplified expression with only the significant terms.



### Simplifying the One-Body Operator

The raw Hamiltonian contains many terms with near-zero coefficients and duplicate Pauli strings.
We apply several operations to simplify it:

1. **apply_threshold()** - Removes terms with coefficients below 1e-10
2. **combine()** - Consolidates identical Pauli strings by adding their coefficients
3. **apply_threshold()** - Removes any new terms with negligible coefficients
4. **sort()** - Orders terms by their coefficients' absolute values.

The resulting operator is a more compact representation of the form:

$$\hat{H}_1 = \sum_j c_j \hat{P}_j$$

Where:
- $c_j$ is the real coefficient for each term
- $\hat{P}_j$ is a Pauli string (product of I, X, Y, Z operators)

This more efficient representation helps us implement the Hamiltonian on quantum hardware with fewer gates.


(-1.73+0.00j)*IIII + (0.62+0.00j)*IZII + (0.62+0.00j)*IIIZ + (0.24+0.00j)*ZIII + (0.24+0.00j)*IIZI


We see many Pauli strings with 0 coefficient as well as many repeated strings. We can now exploit `combine()` and `apply_threshold()`. Since there is many 0 terms already we do `apply_threshold()` first, then `combine()` and `apply_threshold()` again if there was any cancellations. We can finish with `sort()` for neat presentation.

In [40]:
one_body_operator = one_body_operator.apply_threshold().combine().apply_threshold().sort()
print(one_body_operator)

# Display mathematical explanation of the simplified operator
from IPython.display import display, Markdown
display(Markdown(r"""
*The simplified one-body operator after applying thresholds and combining like terms:*

The apply_threshold() method removes Pauli terms with very small coefficients to reduce complexity.
The combine() method groups together like Pauli strings and sums their coefficients.
The sort() method orders the terms by their coefficients' absolute values.

The resulting operator is a more compact representation of the form:

$$\hat{H}_1 = \sum_j c_j \hat{P}_j$$

Where:
- $c_j$ is the real coefficient for each term
- $\hat{P}_j$ is a Pauli string (product of I, X, Y, Z operators)

This more efficient representation helps us implement the Hamiltonian on quantum hardware with fewer gates.
"""))

(-1.73+0.00j)*IIII + (0.62+0.00j)*IZII + (0.62+0.00j)*IIIZ + (0.24+0.00j)*ZIII + (0.24+0.00j)*IIZI


(-1.73+0.00j)*IIII + (0.62+0.00j)*IZII + (0.62+0.00j)*IIIZ + (0.24+0.00j)*ZIII + (0.24+0.00j)*IIZI



*The simplified one-body operator after applying thresholds and combining like terms:*

The apply_threshold() method removes Pauli terms with very small coefficients to reduce complexity.
The combine() method groups together like Pauli strings and sums their coefficients.
The sort() method orders the terms by their coefficients' absolute values.

The resulting operator is a more compact representation of the form:

$$\hat{H}_1 = \sum_j c_j \hat{P}_j$$

Where:
- $c_j$ is the real coefficient for each term
- $\hat{P}_j$ is a Pauli string (product of I, X, Y, Z operators)

This more efficient representation helps us implement the Hamiltonian on quantum hardware with fewer gates.


You should get :

<code>
(-1.73+0.00j)*IIII + (0.62+0.00j)*IIIZ + (0.62+0.00j)*IZII + (0.24+0.00j)*IIZI + (0.24+0.00j)*ZIII
</code>

### Two body term
The two body Hamiltonian is of the following form :

\begin{align*}
    \hat{H}_2 = \frac{1}{2}\sum_{i,j} h_{ijkl} \hat{a}_i^\dagger\hat{a}_j^\dagger \hat{a}_k\hat{a}_l
\end{align*} 

You can now implement the `build_two_body_qubit_hamiltonian` function in the `mapping.py` file. 
Counting all Pauli strings you should produce 4096 Pauli strings.

In [42]:
# Display mathematical explanation of the two-body operator transformation
display(Markdown(r"""
### Two-Body Operator Transformation

The two-body term in the fermionic Hamiltonian represents electron-electron interactions:

$$\hat{H}_2 = \frac{1}{2}\sum_{i,j,k,l} h_{ijkl} \hat{a}_i^\dagger\hat{a}_j^\dagger \hat{a}_k\hat{a}_l$$

Where $h_{ijkl}$ are the two-electron integrals, and the Jordan-Wigner mapping transforms this into:

1. Map each creation operator: $\hat{a}_i^\dagger \rightarrow \frac{1}{2}(\hat{X}_i - i\hat{Y}_i)\prod_{j<i}\hat{Z}_j$
2. Map each annihilation operator: $\hat{a}_i \rightarrow \frac{1}{2}(\hat{X}_i + i\hat{Y}_i)\prod_{j<i}\hat{Z}_j$
3. Multiply these transformed operators according to the fermionic Hamiltonian
"""))

%autoreload
from quantum_chemistry.mapping import build_two_body_qubit_hamiltonian, creation_annihilation_operators_with_jordan_wigner
from quantum_chemistry.molecule.h2_molecule import load_h2_spin_orbital_integral

display(Markdown(r"""
### Building the Two-Body Qubit Hamiltonian

The two-body term represents electron-electron interactions in the fermionic Hamiltonian:

$$\hat{H}_2 = \frac{1}{2}\sum_{i,j,k,l} h_{ijkl} \hat{a}_i^{\dagger}\hat{a}_j^{\dagger} \hat{a}_k\hat{a}_l$$

Where:
- $h_{ijkl}$ are the two-electron integrals from molecular orbital calculations
- $\hat{a}_i^{\dagger}$ and $\hat{a}_j^{\dagger}$ are creation operators
- $\hat{a}_k$ and $\hat{a}_l$ are annihilation operators

Each fermionic operator is mapped to a sum of Pauli strings using the Jordan-Wigner transformation, 
resulting in a quantum Hamiltonian with approximately 4096 terms before simplification.
"""))

distance, one_body, two_body, nuc_eneg = load_h2_spin_orbital_integral("../h2_data","h2_mo_integrals_d_0750.npz")
creation_operators, annihilation_operators = creation_annihilation_operators_with_jordan_wigner(4)

two_operator = build_two_body_qubit_hamiltonian(two_body, creation_operators, annihilation_operators)

# Display analysis of generated two-body operator
display(Markdown(r"""
The raw two-body operator contains approximately 4096 Pauli terms before simplification!

This complexity comes from:
- Each fermionic operator maps to 2 Pauli terms
- Each two-body integral term has 4 fermionic operators
- Therefore each $h_{ijkl}$ term produces $2^4 = 16$ Pauli terms
- With the two-body integral tensor having dimensions $4×4×4×4 = 256$ elements
- This gives $256 × 16 = 4096$ raw Pauli terms

Next, we'll simplify this operator by combining like terms and removing negligible contributions.
"""))


### Two-Body Operator Transformation

The two-body term in the fermionic Hamiltonian represents electron-electron interactions:

$$\hat{H}_2 = \frac{1}{2}\sum_{i,j,k,l} h_{ijkl} \hat{a}_i^\dagger\hat{a}_j^\dagger \hat{a}_k\hat{a}_l$$

Where $h_{ijkl}$ are the two-electron integrals, and the Jordan-Wigner mapping transforms this into:

1. Map each creation operator: $\hat{a}_i^\dagger \rightarrow \frac{1}{2}(\hat{X}_i - i\hat{Y}_i)\prod_{j<i}\hat{Z}_j$
2. Map each annihilation operator: $\hat{a}_i \rightarrow \frac{1}{2}(\hat{X}_i + i\hat{Y}_i)\prod_{j<i}\hat{Z}_j$
3. Multiply these transformed operators according to the fermionic Hamiltonian



### Building the Two-Body Qubit Hamiltonian

The two-body term represents electron-electron interactions in the fermionic Hamiltonian:

$$\hat{H}_2 = \frac{1}{2}\sum_{i,j,k,l} h_{ijkl} \hat{a}_i^{\dagger}\hat{a}_j^{\dagger} \hat{a}_k\hat{a}_l$$

Where:
- $h_{ijkl}$ are the two-electron integrals from molecular orbital calculations
- $\hat{a}_i^{\dagger}$ and $\hat{a}_j^{\dagger}$ are creation operators
- $\hat{a}_k$ and $\hat{a}_l$ are annihilation operators

Each fermionic operator is mapped to a sum of Pauli strings using the Jordan-Wigner transformation, 
resulting in a quantum Hamiltonian with approximately 4096 terms before simplification.


Loading h2_mo_integrals_d_0750.npz



The raw two-body operator contains approximately 4096 Pauli terms before simplification!

This complexity comes from:
- Each fermionic operator maps to 2 Pauli terms
- Each two-body integral term has 4 fermionic operators
- Therefore each $h_{ijkl}$ term produces $2^4 = 16$ Pauli terms
- With the two-body integral tensor having dimensions $4×4×4×4 = 256$ elements
- This gives $256 × 16 = 4096$ raw Pauli terms

Next, we'll simplify this operator by combining like terms and removing negligible contributions.


Applying the `combine` and `apply_threshold` this reduce to 15 Pauli strings!

In [ ]:
# Process the two-body operator by applying threshold, combining like terms, and sorting
two_operator = two_operator.apply_threshold().combine().apply_threshold().sort()
print(two_operator)

# Display mathematical explanation and interpretation of the simplified two-body operator
display(Markdown(r"""
### Simplified Two-Body Operator

We've transformed the raw two-body operator (with ~4096 terms) into this compact representation with only 15 terms!

This dramatic reduction occurs because:
1. Many terms have coefficients that are effectively zero
2. Many Pauli strings appear multiple times with different coefficients that combine
3. Some terms cancel each other out completely

The simplified operator captures the essential physics of electron-electron interactions
while being much more feasible to implement on a quantum computer.

Each term in this expression represents a specific type of interaction in the molecular system.
For example, ZZZZ terms represent pair-wise interactions between electron spins.
"""))

(0.91+0.00j)*IIII
(-0.46+0.00j)*ZIII
(-0.46+0.00j)*IIZI
(-0.45+0.00j)*IZII
(-0.45+0.00j)*IIIZ
(0.17+0.00j)*ZIZI
(0.17+0.00j)*IZIZ
(0.17+0.00j)*IZZI
(0.17+0.00j)*ZIIZ
(0.12+0.00j)*IIZZ
(0.12+0.00j)*ZZII
(0.05+0.00j)*XXXX
(0.05+0.00j)*YYXX
(0.05+0.00j)*XXYY
(0.05+0.00j)*YYYY


You should get :

<code>
(1.83+0.00j)*IIII<br>
(-0.92+0.00j)*ZIII<br>
(-0.92+0.00j)*IIZI<br>
(-0.91+0.00j)*IIIZ<br>
(-0.91+0.00j)*IZII<br>
(0.35+0.00j)*ZIZI<br>
(0.34+0.00j)*IZIZ<br>
(0.33+0.00j)*ZIIZ<br>
(0.33+0.00j)*IZZI<br>
(0.24+0.00j)*IIZZ<br>
(0.24+0.00j)*ZZII<br>
(0.09+0.00j)*YYYY<br>
(0.09+0.00j)*XXYY<br>
(0.09+0.00j)*YYXX<br>
(0.09+0.00j)*XXXX<br>
</code>

### Molecular Hamiltonian

The molecular Hamiltonian is just the sum of the one and two body terms. Implement th function `build_qubit_hamiltonian` in `mapping.py`. You should now be able to run this code.

In [43]:
%autoreload
from quantum_chemistry.mapping import build_qubit_hamiltonian, creation_annihilation_operators_with_jordan_wigner
from quantum_chemistry.molecule.h2_molecule import load_h2_spin_orbital_integral

distance, one_body, two_body, nuc_eneg = load_h2_spin_orbital_integral("../h2_data","h2_mo_integrals_d_0750.npz")
creation_operators, annihilation_operators = creation_annihilation_operators_with_jordan_wigner(4)

qubit_hamiltonian = build_qubit_hamiltonian(one_body, two_body, creation_operators, annihilation_operators)

print(qubit_hamiltonian)

Loading h2_mo_integrals_d_0750.npz
(-0.82+0.00j)*IIII
(-0.22+0.00j)*ZIII
(-0.22+0.00j)*IIZI
(0.17+0.00j)*ZIZI
(0.17+0.00j)*IIIZ
(0.17+0.00j)*IZII
(0.17+0.00j)*IZIZ
(0.17+0.00j)*IZZI
(0.17+0.00j)*ZIIZ
(0.12+0.00j)*IIZZ
(0.12+0.00j)*ZZII
(0.05+0.00j)*XXXX
(0.05+0.00j)*YYXX
(0.05+0.00j)*XXYY
(0.05+0.00j)*YYYY


In [44]:
qubit_hamiltonian = build_qubit_hamiltonian(one_body, two_body, creation_operators, annihilation_operators)

# Display explanation of complete molecular Hamiltonian
from IPython.display import display, Markdown
display(Markdown(r"""
### Complete Molecular Hamiltonian

The full molecular Hamiltonian combines:

1. **One-body terms** (~5 significant terms) - Represent electron-nuclei interactions and 
   electron kinetic energy
2. **Two-body terms** (~15 significant terms) - Represent electron-electron interactions
3. **Nuclear repulsion** (constant term) - Added to the final energy but not to the operator

This Hamiltonian with ~20 Pauli terms is remarkably simplified from the raw mapping that 
initially produced over 4,000 terms through:
- Term combination (adding coefficients of identical Pauli strings)
- Threshold application (removing negligible terms)
- Natural cancellations from the Jordan-Wigner mapping

The simplified operator can be directly used in quantum algorithms like VQE.
"""))

print(qubit_hamiltonian)


### Complete Molecular Hamiltonian

The full molecular Hamiltonian combines:

1. **One-body terms** (~5 significant terms) - Represent electron-nuclei interactions and 
   electron kinetic energy
2. **Two-body terms** (~15 significant terms) - Represent electron-electron interactions
3. **Nuclear repulsion** (constant term) - Added to the final energy but not to the operator

This Hamiltonian with ~20 Pauli terms is remarkably simplified from the raw mapping that 
initially produced over 4,000 terms through:
- Term combination (adding coefficients of identical Pauli strings)
- Threshold application (removing negligible terms)
- Natural cancellations from the Jordan-Wigner mapping

The simplified operator can be directly used in quantum algorithms like VQE.


(-0.82+0.00j)*IIII
(-0.22+0.00j)*ZIII
(-0.22+0.00j)*IIZI
(0.17+0.00j)*ZIZI
(0.17+0.00j)*IIIZ
(0.17+0.00j)*IZII
(0.17+0.00j)*IZIZ
(0.17+0.00j)*IZZI
(0.17+0.00j)*ZIIZ
(0.12+0.00j)*IIZZ
(0.12+0.00j)*ZZII
(0.05+0.00j)*XXXX
(0.05+0.00j)*YYXX
(0.05+0.00j)*XXYY
(0.05+0.00j)*YYYY


You should get :

<code>
(-0.82+0.00j)*IIII<br>
(-0.22+0.00j)*IIZI<br>
(-0.22+0.00j)*ZIII<br>
(0.17+0.00j)*ZIZI<br>
(0.17+0.00j)*IIIZ<br>
(0.17+0.00j)*IZII<br>
(0.17+0.00j)*IZIZ<br>
(0.17+0.00j)*ZIIZ<br>
(0.17+0.00j)*IZZI<br>
(0.12+0.00j)*IIZZ<br>
(0.12+0.00j)*ZZII<br>
(0.05+0.00j)*YYYY<br>
(0.05+0.00j)*XXYY<br>
(0.05+0.00j)*YYXX<br>
(0.05+0.00j)*XXXX<br>
</code>

## You have completed your first mapping of H2!

What now? The next step is to use this mapping to evaluate the Hamiltonian on a quantum computer. This is the topic of the next tutorial.


Notebook by **Maxime Dion** <maxime.dion@usherbrooke.ca><br>
For the QSciTech-QuantumBC virtual workshop on gate-based quantum computing